In [1]:
import os

instance_uri = "https://api.stage.sinopia.io/resource/6133c0a5-ee94-439a-9f3a-3ec8cf25e1b7"

alma_api_key = os.getenv("ALMA_API_KEY")
uri_region = os.getenv("ALMA_URI_REGION")

In [ ]:
%pip install rdflib
%pip install lxml
%pip install requests

In [3]:
import requests
from rdflib import Graph, Namespace, URIRef
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces
import sys


In [ ]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)

In [ ]:
# Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
bflc = Namespace("http://id.loc.gov/ontologies/bflc/")
# Bind the namespaces to the instance graph
for prefix, url in alma_namespaces:
    instance_graph.bind(prefix, url)
# Get the work URI from the instance graph
work_uri = instance_graph.value(subject=instance_uri, predicate=bf.instanceOf)
# check if the work_uri is none
if work_uri is None:
    print("No work URI found for this instance.")
    sys.exit()

# Ensure work_uri is a URIRef
work_uri = URIRef(work_uri)

# Remove any triples where work_uri is the subject
instance_graph.remove((work_uri, None, None))
# Serialize the instance graph
instance_alma_xml = instance_graph.serialize(format="pretty-xml", encoding="utf-8")

print(instance_alma_xml.decode('utf-8'))

In [ ]:
tree = ET.fromstring(instance_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("xsl/normalize-instance-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
instance_alma_xml = transform(tree)
instance_alma_xml = ET.tostring(
        instance_alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-instance.xml", "wb") as f:
    f.write(instance_alma_xml)
print(instance_alma_xml.decode("utf-8"))

In [ ]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the instance to Alma
def NewInstancetoAlma():
    with open("alma-instance.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=false&stale_version_check=false&cataloger_level=&check_match=false"
                + "&import_profile=&apikey="
                + alma_api_key
            )
            putInstanceToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putInstanceToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewInstancetoAlma()